In [4]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re

In [5]:
#크롤링을 통해 책의 고유 ID를 수집
num = 1
itemids = []
i = 1

#책 id를 추출하는 정규식
compiler1 = re.compile('/Product/Goods/\d{5,8}')
compiler2 = re.compile('\d{5,8}')

while True:
    url = 'http://www.yes24.com/searchcorner/Search?keywordAd=&keyword=&query=%c0%ce%b0%f8%c1%f6%b4%c9&domain=BOOK&page_size=120&PageNumber={}&scode=012'
    page = requests.get(url .format(num))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    bookInfo = soup.select('p.goods_name a')
    if len(bookInfo) <= 0:
        print('Done!')
        break
    
    for info in bookInfo:
        try:
            itemid=compiler1.search(str(info)).group()
            itemids.append(compiler2.search(itemid).group())
        except AttributeError:
            pass

    num += 1
print('총 개수: ', len(itemids))

Done!
총 개수:  808


In [6]:
# DataFrame 생성
columns = ['제목','부제', '저자', '분야', '출판일', '출판사', '가격', '목차', '소개', '출판사제공', '판매지수'] #데이터 프레임의 컬럼명 
dataFrame = pd.DataFrame(columns=columns) #dataFrame을 만들고 지정한 column명으로 설정

In [8]:
# 책의 상세 페이지에 접근하여 데이터 크롤링
n = 0 #dataframe index 초기화
year = 2012  #검색 기간

for i in itemids:
    url = 'http://www.yes24.com/Product/Goods/{}?scode=032&OzSrank=1'
    page = requests.get(url .format(i))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #출간일
    try:
        date = soup.find('span', attrs={'class':'gd_date'}).text
        date = re.sub('년|월|일|\ ', '', date) #한글 단위를 모두 제거
        if int(date[:4]) < year:
            continue
    except AttributeError:
        continue
    
    
    title = soup.find('h2', attrs={'class':'gd_name'}).text #제목을 가져온다.
    if '세트' in title:
        continue
    try:
        sub_title = soup.find('h3', attrs={'class':'gd_nameE'}).text  #부제목을 가져온다.
    except AttributeError:
        sub_title = None
        
    try:
        author = soup.select('span.gd_auth a')[0].text #저자 정보
    except IndexError:
        try:
            author = soup.select('span.gd_auth')[0].text #저자 정보
            author = re.sub('\\r|\\n|\ ', '', author)
        except IndexError:
            author = None
    try:        
        feild = soup.select('a[href*="/24/Category/Display"]')[136].text  #분야를 가져온다.
        if feild == '국내도서':
            feild = soup.select('a[href*="/24/Category/Display"]')[137].text
        
    except IndexError:
        feild = 미분류
    

    publisher = soup.select('span.gd_pub')[0].text #출판사

    #가격
    price = soup.find('em', attrs={'class':'yes_m'}).text
    price = re.sub(',|원', '', price)
    
    #목차
    try:
        contents = soup.select('textarea.txtContentText')[1].text
    except IndexError:
        try:
            contents = soup.select('textarea.txtContentText')[0].text
        except IndexError:
            continue
    
    introduction = soup.select('textarea.txtContentText')[0].text #책소개
    
    #출판사 소개
    try:
        introduction_2 = soup.select('div#infoset_pubReivew')[0].text 
        introduction_2 = re.sub('출판사 리뷰|\\n|펼쳐보기접어보기', ' ', introduction_2) #불용어 제거
    except IndexError:
        introduction_2 = None
    
    sellnum = soup.find('span', attrs={'class':'gd_sellNum'}) #판매지수
    if sellnum is not None:
        compiler = re.compile('\d+')
        sullnum = compiler.search(sellnum.text).group()
    
    
    #['제목','부제', '저자', '분야', '출판일', '출판사', '가격', '목차', '소개', '출판사제공']
    dataFrame.loc[n] = [title, sub_title, author, feild, date, publisher, price, contents, introduction, introduction_2, sellnum]
    n+= 1;

print('데이터 개수: ', len(dataFrame))
print('Done!')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [14]:
dataFrame.to_csv(r'AI_book_data.csv', index=False, sep=',', encoding='utf-8')

In [12]:
n = len(dataFrame)+1

for i in range(n, len(itemids)) :
    url = 'http://www.yes24.com/Product/Goods/{}?scode=032&OzSrank=1'
    page = requests.get(url .format(itemids[i]))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #출간일
    try:
        date = soup.find('span', attrs={'class':'gd_date'}).text
        date = re.sub('년|월|일|\ ', '', date) #한글 단위를 모두 제거
        if int(date[:4]) < year:
            continue
    except AttributeError:
        continue
    
    
    title = soup.find('h2', attrs={'class':'gd_name'}).text #제목을 가져온다.
    if '세트' in title:
        continue
    try:
        sub_title = soup.find('h3', attrs={'class':'gd_nameE'}).text  #부제목을 가져온다.
    except AttributeError:
        sub_title = None
        
    try:
        author = soup.select('span.gd_auth a')[0].text #저자 정보
    except IndexError:
        try:
            author = soup.select('span.gd_auth')[0].text #저자 정보
            author = re.sub('\\r|\\n|\ ', '', author)
        except IndexError:
            author = None
    try:        
        feild = soup.select('a[href*="/24/Category/Display"]')[136].text  #분야를 가져온다.
        if feild == '국내도서':
            feild = soup.select('a[href*="/24/Category/Display"]')[137].text
        
    except IndexError:
        feild = '미분류'
    

    publisher = soup.select('span.gd_pub')[0].text #출판사

    #가격
    price = soup.find('em', attrs={'class':'yes_m'}).text
    price = re.sub(',|원', '', price)
    
    #목차
    try:
        contents = soup.select('textarea.txtContentText')[1].text
    except IndexError:
        try:
            contents = soup.select('textarea.txtContentText')[0].text
        except IndexError:
            continue
    
    introduction = soup.select('textarea.txtContentText')[0].text #책소개
    
    #출판사 소개
    try:
        introduction_2 = soup.select('div#infoset_pubReivew')[0].text 
        introduction_2 = re.sub('출판사 리뷰|\\n|펼쳐보기접어보기', ' ', introduction_2) #불용어 제거
    except IndexError:
        introduction_2 = None
    
    sellnum = soup.find('span', attrs={'class':'gd_sellNum'}) #판매지수
    if sellnum is not None:
        compiler = re.compile('\d+')
        sullnum = compiler.search(sellnum.text).group()
    
    
    #['제목','부제', '저자', '분야', '출판일', '출판사', '가격', '목차', '소개', '출판사제공']
    dataFrame.loc[n] = [title, sub_title, author, feild, date, publisher, price, contents, introduction, introduction_2, sellnum]
    n+= 1;

print('데이터 개수: ', len(dataFrame))
print('Done!')

데이터 개수:  687
Done!


In [19]:
for i in range(len(dataFrame)):
    try:
        dataFrame.iloc[i]['판매지수'] = re.sub('\\n|\||\\r|\ |판매지수|란\?', '', dataFrame.iloc[i]['판매지수'].text)
    except AttributeError:
        pass

In [18]:
re.sub('\\n|\||\\r|\ |판매지수|란\?', '', dataFrame.iloc[0]['판매지수'].text)

'491301'

In [69]:
print(soup.find('span', attrs={'class':'gd_sellNum'}))

None


In [28]:
a = re.compile('.+\d{9}.*')
for i in soup.select('a[href*="/24/Category/Display/"]'):
    print(a.search(i).group())

AttributeError: 'str' object has no attribute 'text'

In [13]:
n

688

In [20]:
dataFrame

,제목,부제,저자,분야,출판일,출판사,가격,목차,소개,출판사제공,판매지수
0,에이트,인공지능에게 대체되지 않는 나를 만드는 법,이지성,자기계발,20191021,차이정원,17000,\r\n당신에게 묻는다prologue 잡스는 왜 죽기 직전까지 인공지능을 붙잡고 있...,\r\n하버드·스탠퍼드·NASA·구글……아이비리그와 실리콘밸리의 천재들은 인공지능 ...,인류가 단 한 번도 경험하지 못한 시대가 오고 있다!『리딩으로 리드하라...,491301
1,핸즈온 머신러닝,"사이킷런, 케라스, 텐서플로 2를 활용한 머신러닝, 딥러닝 완벽 실무",오렐리앙 제롱,IT 모바일,20200504,한빛미디어,55000,\r\n[PART 1 머신러닝]CHAPTER 1 한눈에 보는 머신러닝1.1 머신러닝...,\r\n머신러닝 전문가로 이끄는 최고의 실전 지침서 텐서플로 2.0을 반영한 풀컬러...,컬러판으로 돌아온 아마존 인공지능 분야 부동의 1위 도서머신러닝 전문가...,19815
2,슈퍼인텔리전스,"경로, 위험, 전략",닉 보스트롬,자연과학,20170405,까치(까치글방),25000,\r\n아직 끝이 나지 않은 이야기 : 참새와 부엉이의 우화서문감사의 글1 과거의 ...,"\r\n2014년 출간 이후 [뉴욕 타임스], 아마존 베스트셀러영미권에서 13만 부...",\r 2016년 알파고와 한국 프로 바둑기사 이세돌의 대국은 전 세계적인...,24222
3,우리 아이 AI,4차산업혁명시대 인공지능 융합교육법,박찬,사회 정치,20200302,다빈치books,24000,\r\n1. 인공지능 시대- 패러다임 전환인공지능의 이해 머신러닝과 딥러닝 활용 핀...,\r\n인공지능 교육에는 패러다임 전환이 필요하다!인공지능 시대의 융합교육법초격차의...,학교와 가정에서 우리 아이들과 함께 인공지능 교육을 시작하세요더 많은 ...,24057
4,십 대가 알아야 할 인공지능과 4차 산업혁명의 미래,"인간의 삶, 사회 그리고 컴퓨터 과학 기술을 함께 보다! 처음 만나는 디지털인문학",전승민,청소년,20180520,팜파스,13000,\r\n들어가는 글Chapter 1 누구도 예측하지 못한 신세계가 열리다 -자동화 ...,\r\n이세돌과 알파고의 대국 이후 다양한 매체에서 ‘바야흐로 4차 산업혁명의 시대...,"\r 인간만의 고유한 가치, 인문학적 감각! 이제 컴퓨터과학과 인문학을 ...",43710
...,...,...,...,...,...,...,...,...,...,...,...
683,왜 로봇의 도덕인가,스스로 판단하는 인공지능 시대에 필요한 컴퓨터 윤리의 모든 것,웬델 월러치,인문,20140530,메디치미디어,21000,\r\n예일 대학교의 생명윤리를 위한 학제간 센터(Interdisciplinary ...,\r\n로봇의 윤리에 대한 포괄적인 안내서 공상과학 소설과 영화를 통해 꾸준히 제...,\r 왜 로봇의 도덕이 필요한가? 현대에는 자동차에서부터 쓰레기통에 ...,None
684,창의성이 뭐길래?,인공지능 시대의 창의성 키우는 방법,장은영,자기계발,20180601,솔과학,15000,"\r\n프롤로그추천사(가나다순)PART 1 창의성은 왜 필요한가1 알파고의 시대, ...","\r\n프롤로그추천사(가나다순)PART 1 창의성은 왜 필요한가1 알파고의 시대, ...",None,12
685,핸즈온 머신러닝,"사이킷런과 텐서플로를 활용한 머신러닝, 딥러닝 실무",오렐리앙 제롱,IT 모바일,20180417,한빛미디어,33000,\r\n1부. 머신러닝1장. 한눈에 보는 머신러닝1.1 머신러닝이란?1.2 왜 머신...,\r\n인공지능 분야에 종사한다면 반드시 읽어야 하는 머신러닝 전문가로 이끄는 최고...,아마존 인공지능 분야 부동의 1위 도서 이 책의 원서는 출간 직후부터 ...,10428
686,파이썬 라이브러리를 활용한 머신러닝,사이킷런 핵심 개발자가 쓴 머신러닝과 데이터과학 실무서,안드레아스 뮐러,IT 모바일,20170701,한빛미디어,30000,\r\nCHAPTER 1 소개1.1 왜 머신러닝인가?__1.1.1 머신러닝으로 풀 ...,\r\n사이킷런 핵심 개발자에게 배우는 머신러닝 이론과 구현 현업에서 머신러닝을 연...,\r 실제 문제에 대한 해법을 찾는 머신러닝 기술자를 위한 본격 머신러닝...,None


In [158]:
title

'인공지능 학습 알고리즘과 응용 솔루션 전문가 과정'

In [161]:
len(re.sub('\\n|\\r|판매지수|\ |\|란\? ','',dataFrame.iloc[0]['판매지수']))

TypeError: expected string or bytes-like object

In [98]:
a=re.compile('\d+')
a.search(dataFrame.iloc[10]['판매지수']).group()

TypeError: expected string or bytes-like object

In [99]:
dataFrame.iloc[10]['판매지수'].group()

'18342'

In [90]:
a.search('asdf12312412asdfa')

<re.Match object; span=(0, 0), match=''>

In [147]:
url = 'http://www.yes24.com/Product/Goods/89959711?scode=032&OzSrank=1'
page = requests.get(url .format(i))
soup = BeautifulSoup(page.text, 'html.parser')
soup.select('a[href*="/24/Category/Display"]')[136]

<a href="/24/Category/Display/001001003">IT 모바일</a>

In [133]:
temp = []
for i in soup.select("div.gd_infoSet a"):
    temp.append(i.find('a', href= re.compile(r'/24/Category/Display/\d*'))) 

In [140]:
title

'소프트웨어 객체의 생애 주기'

In [138]:
soup.select('a[href*="/24/Category/Display"]')[0]


소설/시/희곡
에세이
인문
역사
예술
종교
사회 정치
자연과학
경제 경영
자기계발
인물
유아
어린이
청소년
전집
만화
라이트노벨
여행
요리
육아
가정 살림
건강 취미
잡지
대학교재
국어 외국어 사전
IT 모바일
공무원
수험서 자격증
초등참고서
중고등참고서
EBS교재
ELT 사전
문학 소설
경제 경영
인문 사회
예술 대중문화
취미 라이프스타일
컴퓨터
자연과학
대학교재 전문서
해외잡지
일본도서
중국도서
프랑스도서
유아 어린이 청소년
캐릭터북
초등코스북
학습서
Lexile®
로맨스        
BL            
만화          
소설          
경제 경영     
라이트노벨    
판타지/무협   
에세이 시     
인문          
사회 정치     
자기계발      
역사          
종교          
예술 대중문화 
자연과학      
가정 살림     
건강 취미 여행
어린이 유아   
청소년        
국어 외국어   
IT 모바일     
대학교재      
수험서 자격증 
잡지          
해외원서      
오디오북      
무료eBook     
크레마        
19+           
가요
클래식
POP
ROCK
재즈
OST
뮤직 DVD
뮤직 블루레이
유아/어린이/명상
뉴에이지
종교음악
국악
일본음악
월드뮤직
예약음반
중고음반
스타샵
해외 구매
LP(Vinyl)샵
스페셜샵
예약판매
할인행사
블루레이
애니메이션
액션/SF
드라마/코미디
고전/명화
3D 블루레이
다큐/교육
성인(19)
뮤직 DVD
국내외 TV드라마
닌텐도/게임
비디오/VCD
독립/인디영화
블루레이 해외구매
UEK Zone
스틸북샵
예스24 단독판매
교육(아동/성인)
디자인문구   
필기구/필통  
독서         
디지털/가전  
취미/키덜트  
여행/레저    
패션잡화     
가구/수납    
생활/건강    
홈데코/패브릭
주방/푸드    
유아동       
중고 국내도서
중고 외국도서
직배송 중고도서
중고 음반

In [134]:
temp

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [122]:
soup.select("div.gd_infoSet a")

[<a class="btnC" href="javascript:void(0);" onclick="yes24GU.AddToCartByAreaOptionCheck('series_list');"><span class="bWrap"><em class="txt">카트에 넣기</em></span></a>,
 <a class="btnC" href="javascript:void(0);" onclick="yes24GU.addToMyListByArea('series_list');"><span class="bWrap"><em class="txt">리스트에 넣기</em></span></a>,
 <a href="javascript:void(0);" onclick="toggleSeriesList();"><span class="sort_square"><em class="bgGD">뷰타입 변경</em></span></a>,
 <a href="/Product/Goods/79216048" onclick="setPcodeBySeriesListType();"><img alt="브리태니커 만화 백과 : 이상기후 + 나사와 첨단 우주 과학 세트 " border="0" class="lazy" data-original="http://image.yes24.com/goods/79216048/M" src="//image.yes24.com/momo/Noimg_M.gif"/></a>,
 <a href="/Product/Goods/79216048" onclick="setPcodeBySeriesListType();">브리태니커 만화 백과 : 이상기후 + 나사와 첨단 우주 과학 세트 </a>,
 <a href="/Product/Goods/78907581" onclick="setPcodeBySeriesListType();"><img alt="브리태니커 만화 백과 : 나사와 첨단 우주 과학" border="0" class="lazy" data-original="http://image.yes24.com/goods/78907